In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Classifying Handwritten Digits

In [ ]:
import os
import struct

def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(
        path, f'{kind}-labels-idx1-ubyte'
    )
    images_path = os.path.join(
        path, f'{kind}-images-idx3-ubyte'
    )
    
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        labels = np.fromfile(lbpath, dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII", imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)
        images = ((images / 255.) - .5) * 2

    return images, labels

In [ ]:
# Load dataset into NumPy arrays
X_train, y_train = load_mnist('./MNIST', kind='train')
print(f'Rows: {X_train.shape[0]}, columns: {X_train.shape[1]}')

X_test, y_test = load_mnist('./MNIST', kind='t10k')
print(f'Rows: {X_test.shape[0]}, columns: {X_test.shape[1]}')

In [ ]:
# Plot 10 data samples
fig, ax = plt.subplots(nrows=2, ncols=5, sharex=True, sharey=True)
ax = ax.flatten()
for i in range(10):
    img = X_train[y_train == i][0].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys')

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()

In [ ]:
# Plot 25 samples of 7
fig, ax = plt.subplots(nrows=5, ncols=5, sharex=True, sharey=True)
ax = ax.flatten()
for i in range(25):
    img = X_train[y_train == 7][i].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys')

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()

In [ ]:
# initialize neural network
from multilayer_perceptron import NeuralNetMLP

nn = NeuralNetMLP(
    n_hidden=100, l2=0.01,
    epochs=200, eta=0.0005,
    minibatch_size=100,
    shuffle=True, seed=1
)

In [ ]:
# train neural network
nn.fit(
    X_train=X_train[:55000], y_train=y_train[:55000],
    X_valid=X_train[55000:], y_valid=y_train[55000:]
)

In [ ]:
# plot results
plt.figure(figsize=(10, 10))
plt.plot(range(nn.epochs), nn.eval_['cost'])
plt.ylabel('Cost')
plt.xlabel('Epochs')
plt.show()

In [ ]:
# plot training and validation accuracy
plt.figure(figsize=(10, 10))
plt.plot(range(nn.epochs), nn.eval_['train_acc'], label='training')
plt.plot(range(nn.epochs), nn.eval_['valid_acc'], label='validation', linestyle='--')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend()
plt.show()

In [ ]:
# make predictions on test set
y_test_pred = nn.predict(X_test)

# calculate accuracy
acc = np.sum(y_test == y_test_pred).astype(np.float) / X_test.shape[0]
print(f'Training accuracy: {(acc * 100):.2f}%')

In [ ]:
miscl_img = X_test[y_test != y_test_pred][:25]
correct_lab = y_test[y_test != y_test_pred][:25]
miscl_lab = y_test_pred[y_test != y_test_pred][:25]

fig, ax = plt.subplots(nrows=5, ncols=5, figsize=(10, 10),
                       sharex=True, sharey=True)
ax = ax.flatten()
for i in range(25):
    img = miscl_img[i].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys', interpolation='nearest')
    ax[i].set_title(f'{i+1}) t: {correct_lab[i]} p: {miscl_lab[i]}')

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()